# Youtube API Automation

For the first part, the code fetches publicly available data from any youtube channel of choice,

You can provide as many channel ids as you want. I have chosen 4 channels of my choice as an example for ease.

The next part provides a code to fetch private data and can be used to get insights of your own channel. 

This requires authentication since the data is private.

The data is then fetched everyday using the following code

Both of these datasets are automatically uploaded to this google sheet:
https://docs.google.com/spreadsheets/d/1GYlz0ho4Okj9OvKzyFOpRBFIgINAtKKhHIZ2i6jsFyU/edit?usp=sharing

made with python version 3.11.7

Make sure to install all of these 

pip install pandas

pip install gspread

pip install isodate

pip install schedule

pip install google-api-client

pip install gspread-dataframe

In [53]:
api_key = 'xxxx' #replace with your own api-key. This api key wont work as it is a fake one

modular_filepath = 'D:/Python/Youtube Analytics/modular-command.json' #replace with your own Project ID file path


# you can insert any desired youtube account id
Channel_ID = ['UCC1bgZmrGi8_ALvcfm2_f4Q','UC1itveUZYCaDZZYKpEIe9qw','UCcQhHNoVjqlAogRMljrFrrA','UC_Sxlj9Mb4Tvdm4dYTxZuoA']

In [54]:
import os
import time
import gspread
import schedule
import pandas as pd
from isodate import parse_duration
from googleapiclient.discovery import build
from gspread_dataframe import set_with_dataframe

In [55]:
# make sure to clean privious data in the sheet
GSHEET_NAME = 'Youtube Analytics'

gc = gspread.service_account(filename=modular_filepath)
        
def clear_sheet(GSHEET_NAME):
    sh = gc.open(GSHEET_NAME)
    tab_name = "Sheet0"
    try:
        worksheet = sh.worksheet(tab_name)
        worksheet.clear()
    except gspread.WorksheetNotFound:
        # Create a new worksheet if it doesn't exist
        worksheet = sh.add_worksheet(title=tab_name, rows=1000, cols=26)

clear_sheet(GSHEET_NAME)

In [56]:
channel_dataframes = {}

youtube = build('youtube','v3', developerKey=api_key)


def every_day_data():
    for channel_id in Channel_ID:
        #getting channel info
        def channel_statistics(youtube, channel_id):
            request = youtube.channels().list(
                part = 'snippet,contentDetails,statistics',
                id = channel_id
            )
            response = request.execute()
            return (response['items'])

        channel_response_return = channel_statistics(youtube, channel_id)


        # getting uploads id
        playlist_id = channel_response_return[0]['contentDetails']['relatedPlaylists']['uploads']
        channel_name = channel_response_return[0]["snippet"]["title"]
        
        # getting videos info
        def get_video_list(youtube, playlistid):
            video_list=[]
            request = youtube.playlistItems().list(
                part = 'snippet, contentDetails',
                playlistId = playlistid,
                maxResults = 50
            )
            next_page = True

            while next_page:
                response = request.execute()
                data = response['items']

                for video in data:
                    video_ID = video['contentDetails']['videoId']
                    if video_ID not in video_list:
                        video_list.append(video_ID)

                if 'nextPageToken' in response.keys():
                    next_page = True

                    request = youtube.playlistItems().list(
                    part = 'snippet, contentDetails',
                    playlistId = playlistid,
                    maxResults = 50,
                    pageToken = response['nextPageToken']
                    )
                else:
                    next_page = False

            return video_list

            # fetching data of every single video on the channel

        def get_video_details(youtube, video__list):
            stats_list = []
            batch_size = 50
            for i in range(0,len(video__list)):
                request = youtube.videos().list(
                    part = 'snippet,contentDetails,statistics',
                    id = video__list[i]
                )
                data =request.execute()

                for video in data['items']:
                    title = video['snippet']['title']
                    publish_date = video['snippet']['publishedAt']
                    view_count = video['statistics'].get('viewCount',0)
                    like_count = video['statistics'].get('likeCount',0)
                    dislike_count = video['statistics'].get('dislikeCount',0)
                    comment_count = video['statistics'].get('commentCount',0)
                    share_count = video['statistics'].get('shareCount',0)
                    duration = video['contentDetails'].get('duration',0)
                    video_description = video['snippet']['localized']['description']
                    video_id = video['id']
                    video_url = 'https://www.youtube.com/watch?v='+video['id']
                    
                def get_video_duration(duration_str):
                    duration = parse_duration(duration_str)
                    total_seconds = duration.total_seconds()
                    return total_seconds
                
                video_duration = get_video_duration(duration)
                result = get_video_duration(duration)
                if result < 60:
                    video_type = "Short Video"
                else:
                    video_type = "Regular Video"

                

                stats_dictionary = dict(
                    Channel = channel_name,
                    title = title,
                    video_description = video_description,
                    video_duration = video_duration,
                    video_type = video_type,
                    publish_date = publish_date,
                    view_count = view_count,
                    like_count = like_count,
                    dislike_count = dislike_count,
                    comment_count = comment_count,
                    share_count = share_count,
                    video_id = video_id,
                    video_URL = video_url
                )


                stats_list.append(stats_dictionary)

            return stats_list

        video__list = get_video_list(youtube, playlist_id)
        video_data = get_video_details(youtube, video__list)
        channel_dataframes[channel_id] = pd.DataFrame(video_data)
        display(channel_dataframes[channel_id])
        print('\n Moving on to next channel \n')
        
        
        GSHEET_NAME = 'Youtube Analytics'
        
        gc = gspread.service_account(filename=modular_filepath)
        
        def write_df_to_master_tab(GSHEET_NAME, df):
            sh = gc.open(GSHEET_NAME)
            tab_name = "Sheet0"
            try:
                worksheet = sh.worksheet(tab_name)
            except gspread.WorksheetNotFound:
                # Create a new worksheet if it doesn't exist
                worksheet = sh.add_worksheet(title=tab_name, rows=1000, cols=26)
            last_row = len(worksheet.get_all_values()) + 1
            set_with_dataframe(worksheet, df, row=last_row)

        # Usage:
        write_df_to_master_tab(GSHEET_NAME, channel_dataframes[channel_id])

In [57]:
every_day_data()

,Channel,title,video_description,video_duration,video_type,publish_date,view_count,like_count,dislike_count,comment_count,share_count,video_id,video_URL
0,Faris Shafi,KONG - FARIS SHAFI,WRITTEN AND PERFORMED BY FARIS SHAFI \n\nDIREC...,251.0,Regular Video,2024-02-20T20:01:33Z,254453,28388,0,1983,0,AOA_TKgX85o,https://www.youtube.com/watch?v=AOA_TKgX85o
1,Faris Shafi,With Love - Faris,SONG - WITH LOVE\n\nArtist - Faris Shafi \nhtt...,226.0,Regular Video,2023-01-16T17:22:53Z,1478049,56686,0,2446,0,fcY0W44WgXc,https://www.youtube.com/watch?v=fcY0W44WgXc
2,Faris Shafi,Vitamin D - Faris Shafi x Talal Qureshi,Artist: Faris Shafi - https://www.instagram.co...,270.0,Regular Video,2022-07-27T16:38:17Z,883414,35063,0,2043,0,ZAKgHqHjei0,https://www.youtube.com/watch?v=ZAKgHqHjei0
3,Faris Shafi,HUM - Faris Shafi,Artist: Faris Shafi - https://www.instagram.co...,239.0,Regular Video,2022-07-02T21:37:54Z,518400,29548,0,1611,0,y6buyNH50Wo,https://www.youtube.com/watch?v=y6buyNH50Wo
4,Faris Shafi,LAFZ - Faris Shafi,Song : LAFZ\n\nArtist : Faris Shafi - https://...,265.0,Regular Video,2021-09-12T22:39:01Z,1707629,57001,0,3029,0,scpINT8qiMs,https://www.youtube.com/watch?v=scpINT8qiMs
5,Faris Shafi,INTRODUCTION - FARIS SHAFI,INTRODUCTION - FARIS SHAFI - https://www.insta...,131.0,Regular Video,2021-04-03T16:52:56Z,23807366,637797,0,29503,0,r5Ak4KY8-cs,https://www.youtube.com/watch?v=r5Ak4KY8-cs
6,Faris Shafi,Molotov - Faris Shafi,First 7 seconds of this video are running smoo...,157.0,Regular Video,2021-02-12T13:24:58Z,1394058,43620,0,3383,0,ywToxte1Ppo,https://www.youtube.com/watch?v=ywToxte1Ppo
7,Faris Shafi,Nazar -Faris Shafi,Artist - Faris Shafi - https://www.instagram.c...,283.0,Regular Video,2020-02-07T18:40:16Z,4222478,118087,0,4927,0,JIdyVgRwhkg,https://www.youtube.com/watch?v=JIdyVgRwhkg
8,Faris Shafi,Waasta - Faris Shafi x Ali Sethi,MP3\nhttps://soundcloud.com/farisshafiofficial...,330.0,Regular Video,2018-06-28T13:51:35Z,3079198,62120,0,3586,0,fXJHy5FXzHs,https://www.youtube.com/watch?v=fXJHy5FXzHs
9,Faris Shafi,Waasta - Faris Shafi X Ali Sethi - Teaser,Song: Waasta\nFaris Shafi x Ali Sethi\nMusic :...,24.0,Short Video,2018-06-19T00:01:16Z,106200,2893,0,217,0,YwWjT_yQFIc,https://www.youtube.com/watch?v=YwWjT_yQFIc



 Moving on to next channel 



,Channel,title,video_description,video_duration,video_type,publish_date,view_count,like_count,dislike_count,comment_count,share_count,video_id,video_URL
0,hopeless demon,Dark light,,115.0,Regular Video,2020-08-07T20:41:18Z,33,7,0,1,0,2ZgkDOTUKRE,https://www.youtube.com/watch?v=2ZgkDOTUKRE
1,hopeless demon,Intense 1 vs 4 TDM (sniper only) | PUBG mobile,I lost but it was a good fight. Might have won...,509.0,Regular Video,2020-07-30T15:40:31Z,19,2,0,0,0,u3uGTd-J5FE,https://www.youtube.com/watch?v=u3uGTd-J5FE
2,hopeless demon,Intense fight in Pochinki | Never rush the air...,,643.0,Regular Video,2020-07-16T14:04:57Z,20,2,0,0,0,AxRpdphtN4E,https://www.youtube.com/watch?v=AxRpdphtN4E
3,hopeless demon,Sniper montage PUBG,,132.0,Regular Video,2020-06-27T19:45:16Z,48,4,0,0,0,RImLD0yxmfU,https://www.youtube.com/watch?v=RImLD0yxmfU
4,hopeless demon,Crossbow only challenge | PUBG,First video ever of a crossbow only challenge.,680.0,Regular Video,2020-04-15T18:43:37Z,55,9,0,5,0,b6X5yxLjYv4,https://www.youtube.com/watch?v=b6X5yxLjYv4
5,hopeless demon,Crossbow only. Compilation / montage | PUBG,The first ever crossbow only video on YouTube.,179.0,Regular Video,2020-04-08T15:36:18Z,146,14,0,7,0,FcyWuYb5dN0,https://www.youtube.com/watch?v=FcyWuYb5dN0



 Moving on to next channel 



,Channel,title,video_description,video_duration,video_type,publish_date,view_count,like_count,dislike_count,comment_count,share_count,video_id,video_URL
0,Dego Boop,THE CONCEPT OF PROBABILITY,Hope you guys are having a good day\n\nI thoug...,190.0,Regular Video,2023-12-12T21:05:07Z,5643,854,0,67,0,CnmdufDgF0s,https://www.youtube.com/watch?v=CnmdufDgF0s
1,Dego Boop,INFINITE NIGHT | Short Horror Film,"Two best friends, Sam and Daniel discover an i...",1156.0,Regular Video,2023-11-02T19:45:03Z,34771,4225,0,368,0,wcAsoEi1l64,https://www.youtube.com/watch?v=wcAsoEi1l64
2,Dego Boop,Crazy Karen tried to attack me over some shoes...,,59.0,Short Video,2023-06-12T20:25:10Z,10139,0,0,57,0,GR2iGS1c75o,https://www.youtube.com/watch?v=GR2iGS1c75o
3,Dego Boop,IT COMES AT 3AM,Hope you guys are having a good day\nSorry for...,267.0,Regular Video,2023-05-08T20:00:30Z,23804,3000,0,231,0,wpdJe-jfGxM,https://www.youtube.com/watch?v=wpdJe-jfGxM
4,Dego Boop,HOW TO DRIVE ON THE FLOOR,The long awaited tutorial is finally here\ntha...,503.0,Regular Video,2023-04-03T20:09:33Z,333865,19331,0,1560,0,d_jA2Q8bP5c,https://www.youtube.com/watch?v=d_jA2Q8bP5c
5,Dego Boop,STALKING RANDOM PEOPLES HOUSE PRANK,hope y'all are having a Great Day\nI love all ...,533.0,Regular Video,2023-02-22T21:00:16Z,31687,4266,0,287,0,WaiZNw3Wr90,https://www.youtube.com/watch?v=WaiZNw3Wr90
6,Dego Boop,How friendships start #shorts,,44.0,Short Video,2022-11-16T21:47:05Z,13774,0,0,32,0,M5GlwNOLA7U,https://www.youtube.com/watch?v=M5GlwNOLA7U
7,Dego Boop,Every Moment Matters,Hope y’all enjoy this video I put alot of work...,448.0,Regular Video,2022-11-01T19:30:01Z,37952,4845,0,364,0,E9d2on-iQ7Q,https://www.youtube.com/watch?v=E9d2on-iQ7Q
8,Dego Boop,Five minutes of chaos,"Hello everyone reading this\nMy names Dego, I ...",298.0,Regular Video,2022-10-21T19:30:00Z,51020,5725,0,452,0,3rJRTbJ5grY,https://www.youtube.com/watch?v=3rJRTbJ5grY



 Moving on to next channel 



,Channel,title,video_description,video_duration,video_type,publish_date,view_count,like_count,dislike_count,comment_count,share_count,video_id,video_URL
0,Infinitex,Instru mental,,320.0,Regular Video,2018-05-24T15:21:13Z,43,4,0,0,0,HOevPfB5YB4,https://www.youtube.com/watch?v=HOevPfB5YB4
1,Infinitex,Exotic mashup,,212.0,Regular Video,2018-05-24T13:47:56Z,145,7,0,2,0,RSaqMlnxjpw,https://www.youtube.com/watch?v=RSaqMlnxjpw



 Moving on to next channel 



Execute this to run code everyday 

In [ ]:
schedule.every().day.at('01:00').do(every_day_data)
while True:
    schedule.run_pending()
    time.sleep(1)

# Getting Private data

In [58]:
import os
import time
import gspread
import schedule
import pandas as pd
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
import google_auth_oauthlib.flow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from gspread_dataframe import set_with_dataframe
from google_auth_oauthlib.flow import InstalledAppFlow

In [59]:
API_VERSION = 'v2'
API_SERVICE_NAME = 'youtubeAnalytics'

Scopes = ['https://www.googleapis.com/auth/yt-analytics.readonly']


# add your client secret file path in place of mine
CLIENT_SECRETS_FILE = "D:/Python/Youtube Analytics/client_secret_file.json"

# add your modular file path
modular_filepath = "D:/Python/Youtube Analytics/modular-command.json"

In [60]:
# set the dates for which you need the data
start_date = '2018-07-27'
end_date = '2018-09-01'


ids = 'channel==MINE' # since the data being requested is private so using your own channel is most reasonable
dimensions = 'day' # you can use your own dimension but day suits best for this code
sort = 'day' # similarily day suits best for sorting

In [61]:
# google auth process

def get_service():
    credentials = None
    if os.path.exists("token.json"):
        credentials = Credentials.from_authorized_user_file("token.json", Scopes)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, Scopes)
            credentials = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(credentials.to_json())
    return build(API_SERVICE_NAME, API_VERSION, credentials = credentials)


def execute_api_request(client_library_function, **kwargs):
  response = client_library_function(
    **kwargs
  ).execute()

  print(response)
  return response

In [62]:
#### getting average view duration as an example after authorization is complete

if __name__ == '__main__':
  os.environ['OAUTHLIB_INSECURE_TRANSPORT'] = '1'

  youtubeAnalytics = get_service()
  avg_view_duration_response = execute_api_request(
      youtubeAnalytics.reports().query,
      ids=ids,
      startDate=start_date,
      endDate=end_date,
      metrics='averageViewDuration',
      dimensions=dimensions,
      sort = sort,
  )
  
  
list_avgviewDuration = []
for rows in avg_view_duration_response.get('rows',[]):
    date_value = rows[0]
    avg_view_duration = rows[1]
    
    list_avgviewDuration.append(dict(
      date_value = date_value,
      avg_view_duration = avg_view_duration
    ))
    
df1 = pd.DataFrame(list_avgviewDuration)


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=384672822950-0ujdaqfvboqkbinqsq45840vvkjvb9i4.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58439%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=R4PyQxlfIP0r78fKRSLbQaJIQFtm5n&access_type=offline
{'kind': 'youtubeAnalytics#resultTable', 'columnHeaders': [{'name': 'day', 'columnType': 'DIMENSION', 'dataType': 'STRING'}, {'name': 'averageViewDuration', 'columnType': 'METRIC', 'dataType': 'INTEGER'}], 'rows': [['2018-07-27', 10], ['2018-07-28', 63], ['2018-07-29', 24], ['2018-07-30', 39], ['2018-07-31', 60], ['2018-08-01', 0], ['2018-08-02', 0], ['2018-08-03', 0], ['2018-08-04', 0], ['2018-08-05', 30], ['2018-08-06', 0], ['2018-08-07', 0], ['2018-08-08', 39], ['2018-08-09', 0], ['2018-08-10', 0], ['2018-08-11', 11], ['2018-08-12', 0], ['2018-08-13', 0], ['2018-08-14', 0], ['2018-08-15', 0], ['2018-08-16', 0], ['2018-

In [63]:
### after verification and all this functions is the main function that fetches the data from youtube 

def private_data_function():
        ########estimated Minutes watched

        est_min_watched_response = execute_api_request(
                youtubeAnalytics.reports().query,
                ids=ids,
                startDate=start_date,
                endDate=end_date,
                metrics='estimatedMinutesWatched',
                dimensions=dimensions,
                sort = sort
        )

        est_min_list = []
        for rows in est_min_watched_response.get('rows', []):
                date_value = rows[0]
                est_min_watched = rows[1]
        
                est_min_list.append(dict(
                date_value=date_value,
                est_min_watched=est_min_watched
        ))


        df2 = pd.DataFrame(est_min_list)



        ##### Engagement Matrix

        engagement_list = []

        engagement_response = execute_api_request(
                youtubeAnalytics.reports().query,
                ids=ids,
                startDate=start_date,
                endDate=end_date,
                metrics='views,comments,likes,dislikes,shares,subscribersGained,subscribersLost',
                dimensions=dimensions,
                sort = sort
        )  

        for row in engagement_response.get('rows', []):
                date_value = row[0]
                views = row[1]
                comments = row[2]
                likes = row[3]
                dislikes = row[4]
                shares = row[5]
                subscribers_gained = row[6]
                subscribers_lost = row[7]
                # print(f"Date: {date_value}, Views: {views}, Comments: {comments}, Likes: {likes}, Dislikes:{dislikes}, Shares:{shares}, Subscribers Gained: {subscribers_gained}, Subscribers Lost:{subscribers_lost}")
                
                engagement_list.append(dict(
                        date_value = date_value,
                        views = views,
                        comments = comments,
                        likes = likes,
                        dislikes = dislikes,
                        shares = shares,
                        subscribers_gained = subscribers_gained,
                        subscribers_lost = subscribers_lost
                ))
                
        df3 = pd.DataFrame(engagement_list)


        #averageViewDuration for traffic 

        traffic_list = []

        traffic_response = execute_api_request(
                youtubeAnalytics.reports().query,
                ids='channel==MINE',
                startDate=start_date,
                endDate=end_date,
                metrics='averageViewDuration',
                dimensions=dimensions,
                sort = sort
        )  
        for row in traffic_response.get('rows', []):
                date_value = row[0]
                view_duration = row[1]
                # print(f"Date: {date_value}, Avg View Duration: {view_duration} seconds")

                traffic_list.append(dict(
                        date_value = date_value,
                        traffic_view_duration = view_duration
                ))        

        df4 = pd.DataFrame(traffic_list)

        dd=pd.merge(df2,df3,on='date_value')
        private_data_df=pd.merge(dd,df4, on='date_value')
        display(private_data_df)

        # uploading on a google sheet
        GSHEET_NAME1 = 'Youtube Analytics'
        TAB_NAME1 = 'Private Data'
        range_to_clear = 'Private Data!A1:Z2000'
        gc = gspread.service_account(filename=modular_filepath)
        
        def write_df_to_sheet(GSHEET_NAME, TAB_NAME, df):
                sh = gc.open(GSHEET_NAME)
                worksheet = sh.worksheet(TAB_NAME)
                sh.values_clear(range_to_clear)
                set_with_dataframe(worksheet, df)
                        
        write_df_to_sheet(GSHEET_NAME1, TAB_NAME1, private_data_df)

In [64]:
private_data_function()

{'kind': 'youtubeAnalytics#resultTable', 'columnHeaders': [{'name': 'day', 'columnType': 'DIMENSION', 'dataType': 'STRING'}, {'name': 'estimatedMinutesWatched', 'columnType': 'METRIC', 'dataType': 'INTEGER'}], 'rows': [['2018-07-27', 0], ['2018-07-28', 7], ['2018-07-29', 1], ['2018-07-30', 0], ['2018-07-31', 4], ['2018-08-01', 0], ['2018-08-02', 0], ['2018-08-03', 0], ['2018-08-04', 0], ['2018-08-05', 0], ['2018-08-06', 0], ['2018-08-07', 0], ['2018-08-08', 2], ['2018-08-09', 0], ['2018-08-10', 0], ['2018-08-11', 0], ['2018-08-12', 0], ['2018-08-13', 0], ['2018-08-14', 0], ['2018-08-15', 0], ['2018-08-16', 0], ['2018-08-17', 0], ['2018-08-18', 0], ['2018-08-19', 3], ['2018-08-20', 0], ['2018-08-21', 0], ['2018-08-22', 0], ['2018-08-23', 0], ['2018-08-24', 0], ['2018-08-25', 0], ['2018-08-26', 0], ['2018-08-27', 0], ['2018-08-28', 0], ['2018-08-29', 0], ['2018-08-30', 0], ['2018-08-31', 0], ['2018-09-01', 0]]}
{'kind': 'youtubeAnalytics#resultTable', 'columnHeaders': [{'name': 'day', 'c

,date_value,est_min_watched,views,comments,likes,dislikes,shares,subscribers_gained,subscribers_lost,traffic_view_duration
0,2018-07-27,0,1,0,0,0,0,0,0,10
1,2018-07-28,7,7,3,2,0,2,1,0,63
2,2018-07-29,1,3,0,0,0,1,0,0,24
3,2018-07-30,0,1,0,0,0,0,0,0,39
4,2018-07-31,4,4,0,1,0,0,0,0,60
5,2018-08-01,0,0,0,-1,0,0,0,0,0
6,2018-08-02,0,0,0,0,0,0,0,0,0
7,2018-08-03,0,0,0,0,0,0,0,0,0
8,2018-08-04,0,0,0,0,0,0,0,0,0
9,2018-08-05,0,1,0,0,0,0,0,0,30


Run this to fetch private data everyday

In [ ]:
schedule.every().day.at('01:00').do(private_data_function)
while True:
    schedule.run_pending()
    time.sleep(1)